In [1]:
import os 
import json
import pickle
import datetime

import pandas as pd
from pprint import pprint
from bisect import bisect_left as find_prev
import matplotlib.pyplot as plt

import xml.etree.ElementTree as xml

strptime = datetime.datetime.strptime

In [2]:
episodes_numbers = list(map(str, range(175, 185+1)))
videos_data      = xml.parse('eastenders.collection.xml').getroot().findall("./VideoFile")

episodes_filenames = {v.find('id').text : v.find('filename').text
                      for v in videos_data
                      if v.find('id').text in episodes_numbers}
episodes_filenames

{'175': '5531550228324592939.mp4',
 '176': '5534228999422914578.mp4',
 '177': '5539381671692122744.mp4',
 '178': '5542003749222140011.mp4',
 '179': '5544574287152993687.mp4',
 '180': '5544620672795594434.mp4',
 '181': '5547193787702629969.mp4',
 '182': '5549784941472309008.mp4',
 '183': '5552368364300855101.mp4',
 '184': '5555325449284154780.mp4',
 '185': '5555360238519252381.mp4'}

In [69]:
shots_data = []

for line in open('eastenders.masterShotReferenceTable.txt', 'r'):
    line = line.strip()
    if not line: continue
    file = line.split()[0].strip()
    if file in episodes_numbers:
        file = episodes_filenames[file]
        if file not in shots_starts: 
            shots_starts[file] = []
            shots_ends[file] = []
            shots_ids[file] = []
        _, shot_id, start, end = line.split()
        ref = datetime.datetime(1900, 1, 1)
        start_s = (strptime(start[1:12], '%H:%M:%S:%f') - ref).total_seconds()
        end_s   = (strptime(end[1:12], '%H:%M:%S:%f') - ref).total_seconds()
        duration = end_s - start_s

        shots_data.append((file, shot_id.split('_')[1], start[1:12], end[1:12], start_s, end_s, duration))

In [70]:
df_shots = pd.DataFrame(shots_data, columns=['file', 'shot_id', 'start', 'end', 'start_s', 'end_s', 'duration'])

In [71]:
df_shots

,file,shot_id,start,end,start_s,end_s,duration
0,5531550228324592939.mp4,1,00:00:00:00,00:00:00:12,0.00,0.12,0.12
1,5531550228324592939.mp4,2,00:00:00:13,00:00:00:24,0.13,0.24,0.11
2,5531550228324592939.mp4,3,00:00:01:00,00:00:01:18,1.00,1.18,0.18
3,5531550228324592939.mp4,4,00:00:01:19,00:00:02:05,1.19,2.05,0.86
4,5531550228324592939.mp4,5,00:00:02:06,00:00:24:13,2.06,24.13,22.07
...,...,...,...,...,...,...,...
21207,5555360238519252381.mp4,1732,01:27:01:16,01:27:06:10,5221.16,5226.10,4.94
21208,5555360238519252381.mp4,1733,01:27:06:11,01:27:07:13,5226.11,5227.13,1.02
21209,5555360238519252381.mp4,1734,01:27:07:14,01:27:09:08,5227.14,5229.08,1.94
21210,5555360238519252381.mp4,1735,01:27:09:09,01:27:18:03,5229.09,5238.03,8.94


In [72]:
df_shots.to_csv('segments_timing.csv')

In [88]:
facerec_data = []
for file in os.listdir('../facerec_out/3.coref_people/'):
     for e in json.load(open('../facerec_out/3.coref_people/' + file, 'rb')):
            facerec_data.append((file.replace('json', 'mp4'), e['name'], e['npt'], e['confidence']))

df_facerec = pd.DataFrame(facerec_data, columns=['file', 'character', 'time_s', 'confidence'])

In [89]:
df_facerec

,file,character,time_s,confidence
0,5555360238519252381.mp4,Lauren Branning,61.0,0.336553
1,5555360238519252381.mp4,Stacey Branning,64.0,0.324807
2,5555360238519252381.mp4,Julie Perkins,66.0,0.160653
3,5555360238519252381.mp4,Julie Perkins,67.0,0.174376
4,5555360238519252381.mp4,Julie Perkins,114.0,0.262758
...,...,...,...,...
63536,5539381671692122744.mp4,Julie Perkins,7175.0,0.363719
63537,5539381671692122744.mp4,Julie Perkins,7177.0,0.209327
63538,5539381671692122744.mp4,Julie Perkins,7178.0,0.201113
63539,5539381671692122744.mp4,Julie Perkins,7179.0,0.232778


In [90]:
df_facerec['character'].unique()

array(['Lauren Branning', 'Stacey Branning', 'Julie Perkins', 'Kat Moon',
       'Alfie Moon', 'Janine Malloy', 'Ryan Malloy', 'Pat Evans',
       'Jean Slater', 'Max Branning', 'Phil Mitchell', 'Roxy Mitchell',
       'Billy Mitchell'], dtype=object)

In [91]:
df_shots

,file,shot_id,start,end,start_s,end_s,duration
0,5531550228324592939.mp4,1,00:00:00:00,00:00:00:12,0.00,0.12,0.12
1,5531550228324592939.mp4,2,00:00:00:13,00:00:00:24,0.13,0.24,0.11
2,5531550228324592939.mp4,3,00:00:01:00,00:00:01:18,1.00,1.18,0.18
3,5531550228324592939.mp4,4,00:00:01:19,00:00:02:05,1.19,2.05,0.86
4,5531550228324592939.mp4,5,00:00:02:06,00:00:24:13,2.06,24.13,22.07
...,...,...,...,...,...,...,...
21207,5555360238519252381.mp4,1732,01:27:01:16,01:27:06:10,5221.16,5226.10,4.94
21208,5555360238519252381.mp4,1733,01:27:06:11,01:27:07:13,5226.11,5227.13,1.02
21209,5555360238519252381.mp4,1734,01:27:07:14,01:27:09:08,5227.14,5229.08,1.94
21210,5555360238519252381.mp4,1735,01:27:09:09,01:27:18:03,5229.09,5238.03,8.94


In [92]:
%%time 
shots_data = []
for i, e in df_facerec.iterrows():
    shot = df_shots[(df_shots['file'] == e['file']) &
                    (df_shots['start_s'] <= e['time_s']) & 
                    (df_shots['end_s'] >= e['time_s'])].values
    
    assert(shot.shape == (1, 7))
    shots_data.append(shot[0])

CPU times: user 3min 13s, sys: 44.9 ms, total: 3min 13s
Wall time: 3min 14s


In [93]:
shots_data[:1]

[array(['5555360238519252381.mp4', '18', '00:00:59:04', '00:01:03:15',
        59.04, 63.15, 4.109999999999999], dtype=object)]

In [94]:
df_facerec.head()

,file,character,time_s,confidence
0,5555360238519252381.mp4,Lauren Branning,61.0,0.336553
1,5555360238519252381.mp4,Stacey Branning,64.0,0.324807
2,5555360238519252381.mp4,Julie Perkins,66.0,0.160653
3,5555360238519252381.mp4,Julie Perkins,67.0,0.174376
4,5555360238519252381.mp4,Julie Perkins,114.0,0.262758


In [95]:
df_shots_final = pd.DataFrame(shots_data, columns=df_shots.columns)

In [96]:
df_shots_final['character'] = df_facerec['character']

In [97]:
df_shots_final['confidence'] = df_facerec['confidence']

In [98]:
df_shots_final

,file,shot_id,start,end,start_s,end_s,duration,character,confidence
0,5555360238519252381.mp4,18,00:00:59:04,00:01:03:15,59.04,63.15,4.11,Lauren Branning,0.336553
1,5555360238519252381.mp4,19,00:01:03:16,00:01:05:06,63.16,65.06,1.90,Stacey Branning,0.324807
2,5555360238519252381.mp4,20,00:01:05:07,00:01:08:00,65.07,68.00,2.93,Julie Perkins,0.160653
3,5555360238519252381.mp4,20,00:01:05:07,00:01:08:00,65.07,68.00,2.93,Julie Perkins,0.174376
4,5555360238519252381.mp4,25,00:01:48:02,00:02:01:01,108.02,121.01,12.99,Julie Perkins,0.262758
...,...,...,...,...,...,...,...,...,...
63536,5539381671692122744.mp4,2060,01:59:34:02,01:59:35:00,7174.02,7175.00,0.98,Julie Perkins,0.363719
63537,5539381671692122744.mp4,2063,01:59:36:19,01:59:37:19,7176.19,7177.19,1.00,Julie Perkins,0.209327
63538,5539381671692122744.mp4,2064,01:59:37:20,01:59:41:19,7177.20,7181.19,3.99,Julie Perkins,0.201113
63539,5539381671692122744.mp4,2064,01:59:37:20,01:59:41:19,7177.20,7181.19,3.99,Julie Perkins,0.232778


In [99]:
len(df_shots_final['shot_id'].unique())

2368

In [100]:
df_shots_final.to_csv('facerec_query_characters_coref.csv')